<a href="https://colab.research.google.com/github/DHBern/dl_2022/blob/main/dhunibe_lod_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Erste Schritte mit Wikidata und SPARQL


SPARQL ist eine Query-Sprache für nach semantischen Aspekten strukturierte, das *Resource Description Framework* (RDF) nutzende Datenbanken. 

Information ist in solchen Datenbanken in Form von Prädikatsaussagen abgespeichert, die sich in Triples der Form `<subject> <predicate> <object>` ausdrücken lässt.

```
         +----------+
         |   Bern   |       <subject>
         +----+-----+
              |
              |
 ist gelegen  |             <predicate>
 in           |
              |
              v
         +----+-----+
         | Schweiz  |       <object>
         +----------+
```

Diese Triplestruktur bildet auch die Grundlage der Abfragemöglichkeiten. Dabei wird die Prädikataussage in einem `WHERE`-Statement abgebildet.

Die Grundstruktur sieht so aus:

```
SELECT * 
WHERE {
  ?subject ?predicate ?object
}
```
Die konkreten Bestandteile einer Query entstammen dabei Normdatenbanken und Ontologien.

Es gibt diverse Tutorials und Anleitungen wie man Informationen aus Wikidata mit SPARQL-Queries finden, gruppieren auswerten usw. kann. Eines der bekannteren findet sich unter https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial.

Im vorliegenden Notebook geht es nicht darum, die Möglichkeiten von SPARQL im Detail kennenzulernen, sondern nur um eine allererste Annäherung.

Das Notebook verwendet die [`wikidata-plain-sparql`-Library](https://github.com/jelleschutter/wikidata-plain-sparql) von Jelle Schutter, Data Science Student an der FHNW, die einen niederschwelligen Zugang ermöglicht.

Sie lässt sich durch Ausführen der nächsten beiden Zellen im Notebook-Kontext initialisieren.

Die beiden untenstehenden Beisiele können beliebig modifiziert werden. Unter https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples finden sich viele weitere Query-Beispiele, die auf die gleiche Weise auch angepasst und ausgeführt werden können.

## Paket installieren

In [ ]:
!pip install wikidata_plain_sparql

In [5]:
import wikidata_plain_sparql as wikidata

## Data Frame

Das standardmässige Rückgabeformat ist eine zweidimensionale Datenstruktur, ähnlich einer Tabelle mit Zeilen und Spalten. Genauer gesagt handelt es sich um sog. Pandas Data Frames, ein gängiges Data Science-Format, das sich mit vielen Werkzeugen/Libraries weiterverarbeiten lässt.

## Erste Abfrage (data frame)

Im ersten Beispiel ermitteln wir Personen mit Geburtsort Bern.

Für die Query sind folgende Angaben nötig (eruierbar über die Suche unter https://www.wikidata.org):

|property|uri|
|---|---|
|instance of|https://www.wikidata.org/wiki/Property:P31|
|human|https://www.wikidata.org/wiki/Q5|
|place of birth|https://www.wikidata.org/wiki/Property:P19|
|located in the administrative territorial entity |https://www.wikidata.org/wiki/Property:P131|
|Bern|https://www.wikidata.org/wiki/Q70|



In [ ]:
wikidata.query('''
#title: Humans born in Bern, Switzerland
SELECT distinct ?item ?itemLabel ?itemDescription ?sitelinks 
WHERE 
{
    ?item wdt:P31 wd:Q5;              # Any instance of a human.
          wdt:P19/wdt:P131* wd:Q70;   # Who was born (P19) in any value (eg. a hospital)
                                      # that has the property of 'administrative area of' (P131) Bern (Q70).
             wikibase:sitelinks ?sitelinks.
   
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
ORDER BY DESC(?sitelinks)
''')

* Welche Persönlichkeiten sind an eurem Heimatort/Wohnort geboren?
* Welche davon erst nach 1950?
   * Tipp: https://www.wikidata.org/wiki/Help:Dates#Filter_dates_inside_sparql_queries

## Zweite Abfrage (Karte)

|Property|URI|
|---|---|
|Federal Palace of Switzerland|https://www.wikidata.org/wiki/Q30911|
|coordinate location|https://www.wikidata.org/wiki/Property:P625|
|instance of|https://www.wikidata.org/wiki/Property:P31|

In [ ]:
wikidata.query('''
#Places within 1km of the Federal Palace of Switzerland
SELECT ?place ?placeLabel ?location ?instanceLabel
WHERE
{
  wd:Q30911 wdt:P625 ?loc .
  SERVICE wikibase:around {
      ?place wdt:P625 ?location .
      bd:serviceParam wikibase:center ?loc .
      bd:serviceParam wikibase:radius "1" .
  }
  OPTIONAL {    ?place wdt:P31 ?instance  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
  BIND(geof:distance(?loc, ?location) as ?dist)
} ORDER BY ?dist
''', view='map')

* Lassen sich die Resultate nach Typ filtern?

### Ferner

Wie man mit solchen Methoden ein kleines Forschungsprojekt durchführen könnte, zeigt https://github.com/hawc2/wikidata bzw. https://colab.research.google.com/github/hawc2/wikidata/blob/main/Querying_and_Visualizing_Wikidata.ipynb. 

Etwas Hintergrund zur Herangehensweise wird im zugehörigen Blog-Post geliefert: https://sites.temple.edu/tudsc/2022/01/24/visualizing-wikidata-using-python-to-analyze-identity-and-representation-in-wikidata-about-black-art-exhibitions/
